# ファイル操作

* カレントディレクトリをホームに変更

In [1]:
chdir(posix_getpwuid(posix_geteuid())["dir"])

true

## ファイル/フォルダ/シンボリックリンクの作成

* Emptyという空フォルダを作成

In [2]:
mkdir("Empty")

true

* Blankという空ファイルを作成

In [3]:
# 既にBlankが存在していたら内容が削除される  
# 削除したくない場合は, "w" を "r" や "a" にする
$io=fopen("Blank","w");
fclose($io);

true

* 一時ディレクトリの場合
```PHP
$t=tempnam(sys_get_temp_dir(),getmypid());
mkdir($t);
$t; # ディレクトリのパス
```

* フォルダEmptyの中にファイルBlankのシンボリックリンクSymlinkを作成

In [4]:
symlink("../Blank","Empty/Symlink")

true

- ハードリンクの場合
```PHP
link("../Blank","Empty/Link")
```

## 書込み
* Untitled.mdというMarkdownファイルを作成して書込み

In [5]:
# 既にUntitled.mdが存在すれば上書きされる
$io=fopen("Untitled.md","w");
fwrite($io,"# Header 1");
fclose($io);

true

## 移動/名称変更

* フォルダEmptyをPackageに名称変更

In [6]:
rename("Empty","Package")

true

* Packageフォルダ内のSymlinkファイルをAliasに名称変更

In [7]:
rename("Package/Symlink","Package/Alias")

true

* Untitled.mdを移動して,名称変更

In [8]:
rename("Untitled.md","Package/Headers.md")

true

## 追記
* Markdownファイルに追記

In [9]:
$io=fopen("Package/Headers.md","a");
fwrite($io,PHP_EOL."## Header 2".PHP_EOL."### Header 3");
fclose($io);

true

## 読込み
* Markdownファイルを読込み

In [10]:
$io=fopen("Package/Headers.md","r");
print fread($io,filesize("Package/Headers.md"));
fclose($io);

# Header 1
## Header 2
### Header 3


true

## 再帰的にフォルダ作成
* フォルダを一気に作成

In [11]:
mkdir("Package/Module/Submodule/Item",0777,TRUE)

true

- `$recursive=TRUE` を使うことで,作成するフォルダItemの上位フォルダModule,Submoduleが存在していなくても,同時に生成される

## ディレクトリの内容を表示

In [12]:
$a=scandir("Package");
# filter で "." や ".." を取り除く
array_filter($a,function($v){ return $v!="." && $v!=".."; });

[
  2 => "Alias",
  3 => "Headers.md",
  4 => "Module",
]

- ワイルドカードを使って,条件を満たすファイルを見つけ出す場合は

```PHP
glob("*.md")
```

## ファイル/フォルダの複製
* BlankとModuleを複製

In [13]:
copy("Blank","Package/Blank");
system("cp -r Package/Module \"Package/Module Copy\"");

""

- PHPには元々再帰的にコピーする手段がないので,自作関数を用意するか,上に示したようにシェルコマンドに頼るしかない。

## ファイル/フォルダの削除

* ファイルBlankを削除

In [14]:
unlink("Blank")

true

* シンボリックリンクAliasを削除

In [15]:
unlink("Package/Alias")

true

* フォルダModuleを削除

In [16]:
system("rm -r Package/Module")

""

- 空のフォルダは `rmdir` で削除できる。空でない場合は削除できない。
- PHPには元々再帰的にコピーする手段がないので,自作関数を用意するか,上に示したようにシェルコマンドに頼るしかない。

## 権限を確認/変更

* 状態を確認する関数

In [17]:
function check($path) {

	print (file_exists($path) ? "存在しています" : "存在していません").PHP_EOL;

	if (is_link($path)) {
		print "シンボリックリンクです".PHP_EOL;
		print "リンク先: ".readlink($path).PHP_EOL;
		if (is_file($path)) print "リンク先はファイルです".PHP_EOL;
		if (is_dir($path)) print "リンク先はフォルダです".PHP_EOL;
	}
	else {
		if (is_file($path)) print "ファイルです".PHP_EOL;
		if (is_dir($path)) print "フォルダです".PHP_EOL;
	}

	if (is_readable($path)) print "読込可能です".PHP_EOL;
	if (is_writable($path)) print "書込可能です".PHP_EOL;
	if (is_executable($path)) print "実行可能です".PHP_EOL;

	if (filesize($path)==0) print "空です".PHP_EOL;

}

* Markdownファイルの現在の状態を確認

In [18]:
check("Package/Headers.md")

存在しています



ファイルです



読込可能です



書込可能です



null

* 読込/書込権限を剥奪

In [19]:
chmod("Package/Headers.md",0044)

true

* 状態を確認

In [20]:
check("Package/Headers.md")

存在しています



ファイルです



null

* 読込/実行権限を付加

In [21]:
chmod("Package/Headers.md",0544)

true

* 状態を確認

In [22]:
check("Package/Headers.md")

存在しています



ファイルです



読込可能です



実行可能です



null

- `chmod` は数値により権限を指定する
	* 4: 読込可能 (Readable)
	* 2: 書込可能 (Writable)
	* 1: 実行可能 (eXecutable)

- これらの和を3つ並べて指定する。  
    1つ目はユーザの権限,2つ目はゲストの権限,3つ目はその他の者の権限である。  
	PHPでは8進数で表記するために,前に `0` を付加する

	e.g. `chmod($somefile,0754)`
	* ユーザは 7 = 4+2+1 だから,読込可能,書込可能,実行可能
	* ゲストは 5 = 4 + 1 だから,読込可能,実行可能
	* その他は 4 だから,読込のみ可能